# Simple ADK Agents

In [ ]:
! pip install google-cloud-aiplatform[adk] --quiet --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.1/218.1 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.7/335.7 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.5/229.5 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# @title Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [2]:
import vertexai

PROJECT_ID = ! gcloud config get-value project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1" # @param {type:"string"}
STAGING_BUCKET = "gs://agent-engine-staging-bucket-anita" # @param {type:"string"}

# define project information manually if the above code didn't work
if PROJECT_ID == "(unset)":
  PROJECT_ID = "qwiklabs-gcp-02-8501559ee091" # @param {type:"string"}

print(PROJECT_ID)

vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=STAGING_BUCKET,
)

qwiklabs-gcp-02-8501559ee091


## Sequential Agents

In [31]:
def log_model_response(callback_context, llm_response):

  if llm_response.content and llm_response.content.parts:
    txt = llm_response.content.parts[0].text
  if txt:
    print(txt)
  return None

In [41]:
# @title Test the agent locally
from google.adk.agents import Agent
from google.adk.tools import google_search
from google.adk.agents import SequentialAgent, LlmAgent

# from google.adk.models.lite_llm import LiteLlm

AGENT_MODEL = "gemini-2.0-flash" # Can be a string for Gemini or a LiteLlm object
greeter_agent = LlmAgent(
    name="greeter_agent_v1",
    model=AGENT_MODEL,
    description="Greets user.",
    instruction="You are a greetings assistant. "
                "You greet the user in a friendly manner"
                " and ask them what they want to do.",

    after_model_callback=log_model_response,
)
google_search_agent = LlmAgent(

    name="search_agent_v1",
    model=AGENT_MODEL,
    description=(
        "You can search Google and return answers from the Internet."
    ),
    instruction=(
        """You are a helpful Chatbot.
        If you don't know something, then Search Google for the answer."""
    ),
    tools=[google_search],
    after_model_callback=log_model_response,
)
critique_agent = LlmAgent(

    name="critique_agent_v1",
    model=AGENT_MODEL,
    description=(
        "You provide constructive criticism."
    ),
    instruction=(
        """You are a helpful critical assistant.
        You provide constructive criticism to the response."""
    ),
    after_model_callback=log_model_response,
)

refine_agent = LlmAgent(

    name="refine_agent_v1",
    model=AGENT_MODEL,
    description=(
        "You provide constructive criticism."
    ),
    instruction=(
        """You are a helpful  assistant.
        You provide a refined response based on the critical feedback."""
    ),
    after_model_callback=log_model_response,
)
main_agent = SequentialAgent(
 name="main_agent",
 description="Provides Answers to Users Questions.",
 #tools=[agent_tool.AgentTool(agent=google_search_agent)],
 sub_agents=[greeter_agent,google_search_agent,critique_agent,refine_agent],

)



In [42]:
# @title Create an App that uses the agent

from vertexai.preview import reasoning_engines
app = reasoning_engines.AdkApp(
    agent=main_agent,
    enable_tracing=False,
)

In [43]:

# @title Send a message to the locally running agent
# @title Create a user session
user_id = "test-user-id"
session = app.create_session(user_id=user_id)

print(session.id)
from IPython.display import Markdown, display



#test google search
for event in app.stream_query(
    user_id=user_id,
    session_id=session.id,
    message="Research about Google's Deepmind and describe some of their contributions",
):
    lastevent = event
display(Markdown(lastevent["content"]["parts"][0]["text"]))



57569ef5-fcc0-4f4f-a7fa-83a51ecd4a3d
Logged Model Response from callback: Hey there! 👋 How can I help you today? Are you looking for information, want to brainstorm something, or perhaps need assistance with a task? Let me know!

Logged Model Response from callback: Google DeepMind, a British artificial intelligence research laboratory and subsidiary of Alphabet Inc., was founded in 2010 and acquired by Google in 2014. In April 2023, it merged with Google AI's Google Brain division to become Google DeepMind. The company is dedicated to advancing the field of AI through rigorous scientific investigation. Their mission is to create an intuitive future where digital technologies benefit humanity.

Here are some of DeepMind's notable contributions:

*   **AlphaGo:** DeepMind is known for developing AlphaGo, an AI program that defeated a world champion Go player in 2016, a feat previously thought to be decades away. This achievement demonstrated AI's capability to handle tasks requiring dee

Google DeepMind, a British artificial intelligence research laboratory and subsidiary of Alphabet Inc., was founded in 2010 and acquired by Google in 2014. In April 2023, it merged with Google AI's Google Brain division to become Google DeepMind. The company is dedicated to advancing the field of AI through rigorous scientific investigation, with a mission to create an intuitive future where digital technologies benefit humanity.

Here are some of DeepMind's notable contributions:

*   **AlphaGo:** DeepMind is renowned for developing AlphaGo, an AI program that defeated a world champion Go player in 2016, a feat previously considered decades away. This achievement demonstrated AI's capability to handle tasks requiring deep strategic thinking.
*   **AlphaFold:** In 2020, DeepMind launched AlphaFold, an AI system that accurately predicts 3D models of protein structures. This breakthrough in protein structure prediction has significantly accelerated advancements in drug discovery, leading to progress in understanding diseases and identifying potential drug targets. For example, AlphaFold has aided research into antibiotic resistance and the development of new treatments for diseases like malaria. It has also greatly accelerated structural biology research, saving researchers time and resources.
*   **WaveNet:** DeepMind created WaveNet, a realistic text-to-speech model trained on speech samples. It incorporates natural-sounding elements and has been used in products like the Google Assistant, enhancing the user experience by providing more natural and human-like voice interactions.
*   **AI for Energy Efficiency:** DeepMind has developed AI systems that manage data center cooling more efficiently, resulting in significant energy savings. Their AI system has helped save around 30% of the energy needed for cooling Google's data centers, which translates to a notable reduction in carbon emissions and contributes to Google's sustainability efforts.
*   **Deep Learning and Convolutional Neural Networks (CNNs):** While not the sole pioneers, DeepMind significantly advanced the application of CNNs, building upon earlier work such as Yann LeCun's LeNet. DeepMind's contributions, particularly their success in image recognition challenges like ImageNet, demonstrated the power of CNNs for complex computer vision tasks. Their work has propelled advancements in areas such as image classification, object detection, and image segmentation.
*   **Natural Language Processing:** DeepMind's work in natural language processing has led to AI systems that can better understand and generate human-like responses. These advancements are helping to create more accurate translation tools and sophisticated conversational assistants, improving communication and information access across languages.
*   **Gemini:** DeepMind developed Gemini, a family of multimodal AI models that can understand and generate text, images, code, and more.
*   **Optimizing App Recommendations:** DeepMind has helped personalize app recommendations in Google Play by using machine learning to find the apps that users are more likely to use and enjoy, enhancing user engagement and satisfaction.

DeepMind is also focused on AI ethics and safety, particularly on areas such as bias mitigation and ensuring the explainability of AI models. They are actively researching methods to reduce bias in algorithms and developing techniques to make AI decision-making processes more transparent and understandable.

It is important to acknowledge some limitations and criticisms. Training large AI models requires significant computational resources and energy, raising concerns about environmental impact. There are also ongoing discussions about the potential for bias in AI systems and the need for careful monitoring and mitigation strategies. Despite these challenges, DeepMind continues to push the boundaries of AI research while also addressing ethical considerations.
